In [ ]:
# Poisson problem on a square [-1,1]x[-1,1] with Dirichlet boundary conditions
# This problem solves Example 1.1.1 of the book of Elman et al.

import numpy as np
import eztfem as ezt
from func import func
from scipy.sparse.linalg import spsolve

In [ ]:
# create mesh
mesh = ezt.quadrilateral2d([40, 40], 'quad4', origin=np.array([-1, -1]),
                            length=np.array([2, 2]))

In [ ]:
# define the problem
elementdof = np.array([[1, 1, 1, 1],
                        [2, 2, 2, 2]], dtype=int).transpose()
problem = ezt.Problem(mesh, elementdof, nphysq=1)

In [ ]:
# user struct for setting problem coefficients, ...
user = ezt.User()
user.coorsys = 0
user.alpha = 1
user.funcnr = 5
user.func = func

In [ ]:
# define Gauss integration and basis functions
shape = 'quad'
user.xr, user.wg = ezt.gauss_legendre(shape, n=2)
user.phi, user.dphi = ezt.basis_function(shape, 'Q1', user.xr)

In [ ]:
# assemble the system matrix and vector
A, f = ezt.build_system(mesh, problem, ezt.poisson_elem, user)

In [ ]:
# define essential boundary conditions (Dirichlet)
iess = ezt.define_essential(mesh, problem, 'curves', [0, 1, 2, 3])

# fill values for the essential boundary conditions
uess = np.zeros([problem.numdegfd])

# apply essential boundary conditions to the system
ezt.apply_essential(A, f, uess, iess)

In [ ]:
# solve the system
u = spsolve(A.tocsr(), f)

In [ ]:
# plot the solution
mesh_pv = ezt.generate_pyvista_mesh(mesh)
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400))

In [ ]:
# maximum value
print('Maximum value ', max(u))